## Collaborative Filtering

Fusion des dataframmes clicks

In [1]:
import pandas as pd
import os
from tqdm import tqdm

# Répertoire contenant les fichiers
click_dir = "../data/clicks/"

# Fusionner tous les fichiers CSV
all_clicks = []

for file in tqdm(os.listdir(click_dir)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(click_dir, file))
        all_clicks.append(df)

df_clicks = pd.concat(all_clicks, ignore_index=True)
print("✅ Fichiers fusionnés :", df_clicks.shape)
df_clicks.drop_duplicates(inplace=True)


100%|██████████| 385/385 [00:04<00:00, 90.16it/s] 


✅ Fichiers fusionnés : (2988181, 12)


Aperçu premières lignes

In [2]:
df_clicks.head()

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2


Vérification des types et valeurs manquantes.

In [3]:
df_clicks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2988181 entries, 0 to 2988180
Data columns (total 12 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   user_id              object
 1   session_id           object
 2   session_start        object
 3   session_size         object
 4   click_article_id     object
 5   click_timestamp      object
 6   click_environment    object
 7   click_deviceGroup    object
 8   click_os             object
 9   click_country        object
 10  click_region         object
 11  click_referrer_type  object
dtypes: object(12)
memory usage: 273.6+ MB


Toutes tes colonnes sont en object, y compris :

user_id

click_article_id

click_timestamp

session_

### Etape suivante : convertir les types

On convertit pas directement click_timestamp en datetime, on préfère crée une nouvelle colonne click_datetime pour avoir les deux infos a portée de main.

In [4]:
# Conversion des colonnes en types numériques et datetime
cols_int = [
    "user_id",
    "session_id",
    "session_start",
    "session_size",
    "click_article_id",
    "click_timestamp",
    "click_environment",
    "click_deviceGroup",
    "click_os",
    "click_country",
    "click_region",
    "click_referrer_type"
]

for col in cols_int:
    df_clicks[col] = pd.to_numeric(df_clicks[col], errors="coerce")

# Vérifier qu'il n'y a pas de valeurs manquantes après conversion
print(df_clicks.isnull().sum())

# Convertir timestamp en datetime
df_clicks["click_datetime"] = pd.to_datetime(df_clicks["click_timestamp"], unit="ms")

# Vérifier la structure après conversion
df_clicks.info()


user_id                0
session_id             0
session_start          0
session_size           0
click_article_id       0
click_timestamp        0
click_environment      0
click_deviceGroup      0
click_os               0
click_country          0
click_region           0
click_referrer_type    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2988181 entries, 0 to 2988180
Data columns (total 13 columns):
 #   Column               Dtype         
---  ------               -----         
 0   user_id              int64         
 1   session_id           int64         
 2   session_start        int64         
 3   session_size         int64         
 4   click_article_id     int64         
 5   click_timestamp      int64         
 6   click_environment    int64         
 7   click_deviceGroup    int64         
 8   click_os             int64         
 9   click_country        int64         
 10  click_region         int64         
 11  click_referrer_type  int64         


Création de la table d'intéraction.

In [5]:
# 1️⃣ Chaque clic compte pour 1 point d'intérêt
df_clicks["event_strength"] = 1

# 2️⃣ Agréger le nombre de clics par utilisateur et article
df_user_item = df_clicks.groupby(
    ["user_id", "click_article_id"]
)["event_strength"].sum().reset_index()

# 3️⃣ Trier pour un aperçu lisible
df_user_item = df_user_item.sort_values(by="event_strength", ascending=False)

# 4️⃣ Afficher les 10 plus grosses interactions
df_user_item.head(10)

,user_id,click_article_id,event_strength
349865,16280,68851,33
349924,16280,237071,33
349969,16280,363925,33
349859,16280,43032,31
349853,16280,38823,30
61867,2520,237807,17
643904,33937,225378,16
643895,33937,96173,16
2373759,188046,69463,13
224039,10188,73431,13


Split train/test temporel.

Le split temporel permet de simuler un scénario réel où l’on prédit les comportements futurs des utilisateurs à partir de leur historique passé. Cela évite la fuite de données et garantit que le modèle n’apprend pas sur des interactions postérieures à la période d’entraînement.

In [6]:
# On repart de df_clicks (lignes brutes avec les timestamps)
# Tri chronologique
df_clicks_sorted = df_clicks.sort_values(by="click_timestamp")

# Calcul de l'index de coupure 80/20
split_index = int(0.8 * len(df_clicks_sorted))

# Split temporel
df_train = df_clicks_sorted.iloc[:split_index]
df_test = df_clicks_sorted.iloc[split_index:]

# Afficher les bornes temporelles
print("Train : de", df_train['click_datetime'].min(), "à", df_train['click_datetime'].max())
print("Test  : de", df_test['click_datetime'].min(), "à", df_test['click_datetime'].max())

# Vérifier les tailles
print("Train size :", len(df_train))
print("Test size :", len(df_test))


Train : de 2017-10-01 03:00:00.026000 à 2017-10-12 21:20:12.384000
Test  : de 2017-10-12 21:20:12.579000 à 2017-11-13 20:04:14.886000
Train size : 2390544
Test size : 597637


### Préparer les données pour la librairie Surprise.

On va créer un dataset compatible avec Surprise :

Format = user_id, item_id, rating

Ici rating = event_strength

Note :
Surprise n’accepte que des ratings positifs (pas de 0), mais ici tout est ≥1, donc pas de souci.

In [7]:
from surprise import Dataset, Reader

# Pour Surprise, on n'a besoin que de ces 3 colonnes
train_data = df_train[["user_id", "click_article_id"]].copy()
train_data["event_strength"] = 1  # Chaque clic vaut 1

# Comme on a potentiellement plusieurs clics par user/article, on les agrège
train_data_agg = train_data.groupby(
    ["user_id", "click_article_id"]
).sum().reset_index()

# Idem pour le test
test_data = df_test[["user_id", "click_article_id"]].copy()
test_data["event_strength"] = 1

test_data_agg = test_data.groupby(
    ["user_id", "click_article_id"]
).sum().reset_index()

# Définir le Reader : Surprise attend un rating_min et rating_max
reader = Reader(rating_scale=(1, train_data_agg["event_strength"].max()))

# Convertir en dataset Surprise
train_dataset = Dataset.load_from_df(
    train_data_agg[["user_id", "click_article_id", "event_strength"]],
    reader
)

# Pour l'évaluation, on créera un testset séparé
print("✅ Dataset Surprise prêt.")

✅ Dataset Surprise prêt.


Entrainer un modèle SVD :

In [8]:
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# On divise le dataset Surprise en train/test (ici 80/20)
trainset, valset = train_test_split(train_dataset, test_size=0.2, random_state=42)

# Initialiser le modèle SVD
svd_model = SVD(n_factors=50, n_epochs=20, verbose=True)

# Entraîner
svd_model.fit(trainset)

# Prédire sur le set de validation
predictions = svd_model.test(valset)

# Évaluer la RMSE
rmse = accuracy.rmse(predictions)

print(f"✅ Modèle SVD entraîné, RMSE = {rmse:.4f}")

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
RMSE: 0.1366
✅ Modèle SVD entraîné, RMSE = 0.1366


### Générer le top 5 des recommandations 

Pour faire ça :

On prend un utilisateur.

On parcourt tous les articles qu’il n’a pas encore cliqués.

On prédit le “rating” de ces articles.

On trie par score décroissant.

On garde les 5 premiers.

In [9]:
import numpy as np

def get_top_n_recommendations(model, user_id, all_item_ids, known_items, n=5):
    """
    model: modèle SVD entraîné
    user_id: identifiant utilisateur
    all_item_ids: liste de tous les articles
    known_items: liste des articles déjà vus par l'utilisateur
    n: nombre de recommandations
    """
    # Articles que l'utilisateur n'a pas encore cliqués
    items_to_predict = [iid for iid in all_item_ids if iid not in known_items]
    
    # Prédire les notes
    predictions = [model.predict(user_id, iid) for iid in items_to_predict]
    
    # Trier par rating décroissant
    predictions.sort(key=lambda x: x.est, reverse=True)
    
    # Retourner les n meilleurs
    top_n = predictions[:n]
    return [(pred.iid, pred.est) for pred in top_n]

# Exemple: générer les recommandations pour un user_id au hasard
all_articles = df_clicks["click_article_id"].unique()
some_user = train_data_agg["user_id"].iloc[0]

# Récupérer les articles déjà cliqués par cet utilisateur
user_clicked_articles = train_data_agg[train_data_agg["user_id"] == some_user]["click_article_id"].tolist()

# Obtenir les recommandations
reco = get_top_n_recommendations(svd_model, some_user, all_articles, user_clicked_articles, n=5)

print("✅ Top 5 recommandations pour l'utilisateur", some_user)
for art_id, score in reco:
    print(f"- Article {art_id} (score estimé: {score:.4f})")

✅ Top 5 recommandations pour l'utilisateur 0
- Article 68851 (score estimé: 2.6776)
- Article 73431 (score estimé: 1.9246)
- Article 225378 (score estimé: 1.8289)
- Article 43032 (score estimé: 1.7261)
- Article 38823 (score estimé: 1.7181)


Test avec diffénrents utilisateurs.

In [17]:
# Exemple : tester plusieurs utilisateurs aléatoires
import numpy as np

user_ids_sample = np.random.choice(train_data_agg["user_id"].unique(), size=5, replace=False)

for user_id in user_ids_sample:
    # Articles déjà cliqués par l'utilisateur
    clicked = train_data_agg[train_data_agg["user_id"] == user_id]["click_article_id"].tolist()
    
    # Recommandations
    reco = get_top_n_recommendations(svd_model, user_id, all_articles, clicked, n=5)
    
    print(f"\n✅ Recommandations pour l'utilisateur {user_id}")
    for art_id, score in reco:
        print(f"- Article {art_id} (score estimé: {score:.4f})")



✅ Recommandations pour l'utilisateur 178107
- Article 68851 (score estimé: 2.9396)
- Article 43032 (score estimé: 2.0164)
- Article 38823 (score estimé: 1.9999)
- Article 73431 (score estimé: 1.8042)
- Article 146230 (score estimé: 1.7560)

✅ Recommandations pour l'utilisateur 193327
- Article 68851 (score estimé: 2.1882)
- Article 73431 (score estimé: 1.9914)
- Article 146230 (score estimé: 1.7919)
- Article 105941 (score estimé: 1.6686)
- Article 225378 (score estimé: 1.6564)

✅ Recommandations pour l'utilisateur 195824
- Article 68851 (score estimé: 2.8331)
- Article 43032 (score estimé: 2.1154)
- Article 225378 (score estimé: 2.0914)
- Article 38823 (score estimé: 2.0493)
- Article 73431 (score estimé: 1.9106)

✅ Recommandations pour l'utilisateur 97816
- Article 68851 (score estimé: 2.7213)
- Article 38823 (score estimé: 1.9084)
- Article 146230 (score estimé: 1.8717)
- Article 73431 (score estimé: 1.7886)
- Article 43032 (score estimé: 1.7123)

✅ Recommandations pour l'utilisate

Pourquoi l’article 68851 est recommandé à presque tous les utilisateurs ?
Plusieurs raisons peuvent expliquer ce comportement du modèle SVD :

1. Effet de popularité
L’article 68851 est probablement l’un des plus cliqués de tout le dataset.
Le modèle SVD apprend des tendances globales et peut considérer cet article comme un "valeur sûre" à recommander par défaut.

2. Biais des données
Si de nombreux utilisateurs interagissent avec les mêmes articles ou si le dataset est déséquilibré, certains articles ressortent fortement, surtout s’ils apparaissent souvent dans la période d'entraînement.

3. Fonctionnement du SVD
Le modèle SVD décompose les interactions utilisateur/article en facteurs latents.
En l’absence de signaux très personnalisés pour un utilisateur, le modèle peut privilégier les articles avec un biais global élevé, donc perçus comme "bons en général".

4. Pas nécessairement une erreur
Ce comportement peut être acceptable si 68851 est effectivement très populaire.
Cependant, cela peut nuire à la diversité des recommandations, ce qui est un aspect à surveiller selon les objectifs du projet.

Conclusion : Ce n’est pas forcément un bug, mais un effet typique des modèles collaboratifs sur des données biaisées ou déséquilibrées. Si besoin, on peut combiner avec du content-based ou appliquer un post-traitement pour plus de diversité.

## Content-base Filtering

Étape 1 – Imports et chargement des embeddings

In [10]:
import pandas as pd
import numpy as np
import pickle

Étape 2 – Charger la matrice des embeddings

In [11]:
# Chemin vers le fichier embeddings
embeddings_path = "../data/articles_embeddings.pickle"

# Chargement
with open(embeddings_path, "rb") as f:
    articles_embeddings = pickle.load(f)

print("✅ Embeddings chargés.")
print("Shape de la matrice embeddings :", articles_embeddings.shape)

✅ Embeddings chargés.
Shape de la matrice embeddings : (364047, 250)


Étape 3 – Charger les métadonnées des articles

In [12]:
# Charger les métadonnées
df_articles = pd.read_csv("../data/articles_metadata.csv")

print("✅ Metadata chargées.")
print("Shape :", df_articles.shape)
df_articles.head()

✅ Metadata chargées.
Shape : (364047, 5)


,article_id,category_id,created_at_ts,publisher_id,words_count
0,0,0,1513144419000,0,168
1,1,1,1405341936000,0,189
2,2,1,1408667706000,0,250
3,3,1,1408468313000,0,230
4,4,1,1407071171000,0,162


Étape 4 – Créer un DataFrame article_id + embedding

In [13]:
# Création d'une DataFrame avec article_id et embeddings
df_embeddings = pd.DataFrame({
    "article_id": df_articles["article_id"].astype(int),
    "embedding": list(articles_embeddings)
})

print("✅ DataFrame embeddings créé.")
df_embeddings.head()

✅ DataFrame embeddings créé.


,article_id,embedding
0,0,"[-0.16118301, -0.95723313, -0.13794445, 0.0508..."
1,1,"[-0.52321565, -0.974058, 0.73860806, 0.1552344..."
2,2,"[-0.61961854, -0.9729604, -0.20736018, -0.1288..."
3,3,"[-0.7408434, -0.97574896, 0.39169782, 0.641737..."
4,4,"[-0.2790515, -0.97231525, 0.68537366, 0.113056..."


### Étape 5 – Fonction de recommandation Content-Based

Voici la fonction complète qui :

Récupère les articles lus par un utilisateur

Calcule la moyenne des embeddings de ces articles

Calcule les similarités cosine avec tous les autres articles

Retourne le top N articles non lus

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

def get_content_based_recommendations(user_id, df_user_clicks, df_embeddings, top_n=5):
    """
    user_id : identifiant utilisateur
    df_user_clicks : DataFrame avec colonnes ['user_id', 'click_article_id']
    df_embeddings : DataFrame avec colonnes ['article_id', 'embedding']
    """
    # Articles déjà cliqués par l'utilisateur
    clicked_articles = df_user_clicks[df_user_clicks["user_id"] == user_id]["click_article_id"].unique()
    
    if len(clicked_articles) == 0:
        print("⚠️ Cet utilisateur n'a cliqué sur aucun article.")
        return []
    
    # Embeddings des articles cliqués
    embeddings_clicked = df_embeddings[df_embeddings["article_id"].isin(clicked_articles)]["embedding"].tolist()
    
    # Moyenne des embeddings
    user_profile = np.mean(embeddings_clicked, axis=0).reshape(1, -1)
    
    # Embeddings de tous les articles
    all_embeddings = np.stack(df_embeddings["embedding"].values)
    
    # Similarité cosine
    similarities = cosine_similarity(user_profile, all_embeddings).flatten()
    
    # Créer un DataFrame avec scores
    df_scores = pd.DataFrame({
        "article_id": df_embeddings["article_id"],
        "similarity": similarities
    })
    
    # Retirer les articles déjà vus
    df_scores = df_scores[~df_scores["article_id"].isin(clicked_articles)]
    
    # Top N
    top_recommendations = df_scores.sort_values(by="similarity", ascending=False).head(top_n)
    
    return top_recommendations


On récupère l'historique des clicks

In [15]:
df_user_clicks = df_clicks[["user_id", "click_article_id"]].copy()

Étape : Générer un exemple de recommandations

In [16]:
# Choisir un user_id présent dans tes clics train
some_user_id = df_user_clicks["user_id"].iloc[0]

# Générer les recommandations
recommendations = get_content_based_recommendations(
    some_user_id,
    df_user_clicks,
    df_embeddings,
    top_n=5
)

print("✅ Recommandations Content-Based pour l'utilisateur", some_user_id)
print(recommendations)

✅ Recommandations Content-Based pour l'utilisateur 0
        article_id  similarity
162235      162235    0.874799
160966      160966    0.848911
162230      162230    0.841757
155943      155943    0.841703
160079      160079    0.841227


J’ai implémenté les deux approches comme demandé. Pour la mise en production, j’ai choisi le Content-Based Filtering car il est plus léger, ne nécessite pas de modèle entraîné à charger, et reste très pertinent avec les bons embeddings. Cela me permet de respecter les contraintes d’un environnement cloud gratuit